In [1]:
import json
from geopy.geocoders import Nominatim

# Load the JSON data
with open("asthma.json") as file:
    data = json.load(file)

# Initialize the geocoder
geolocator = Nominatim(user_agent="area-coordinates")

# Function to get coordinates from an area name
def get_coordinates(area_name):
    location = geolocator.geocode(area_name)
    if location:
        return {"latitude": location.latitude, "longitude": location.longitude}
    return {"latitude": None, "longitude": None}

# Add coordinates to each feature
for feature in data["features"]:
    area_name = feature["properties"]["area_name"].split(" (")[0]  # Extract only the area name
    feature["properties"]["coordinates"] = get_coordinates(area_name)

# Save the updated JSON data
with open("asthma_with_coordinates.json", "w") as file:
    json.dump(data, file, indent=4)



In [6]:
import json

# Define the range for latitude and longitude
lat_range = (-39, -34)
lon_range = (140.96, 150)

# Read the data from the input JSON file
with open("asthma_with_coordinates.json", "r") as file:
    data = json.load(file)

# Filter the data
filtered_data = []
for feature in data.get('features', []):
    coordinates = feature['properties'].get('coordinates', {})
    latitude = coordinates.get('latitude')
    longitude = coordinates.get('longitude')
    
    # Ensure both latitude and longitude are not None before comparing
    if latitude is not None and longitude is not None:
        if lat_range[0] <= latitude <= lat_range[1] and lon_range[0] <= longitude <= lon_range[1]:
            filtered_data.append(feature)

# Save the filtered data to a new JSON file
with open("asthma_with_coordinates_filtered.json", "w") as file:
    json.dump({"features": filtered_data}, file, indent=4)


In [11]:
import requests,json
url="https://gateway.api.epa.vic.gov.au/environmentMonitoring/v1/sites"
params={"environmentalSegment":"air"}
headers={
    'User-agent':'curl/8.4.0',
    'Cache-Control':'no-cache',
    'X-API-Key':'f6694fb4cb45496a816c8b630e885f92',
}
response=requests.get(url,params=params,headers=headers)
data=json.loads(response.text)
for record in sorted(data['records'], key=lambda x: x['siteName']):
    # print(record['siteName'])
    # print(record['geometry']['coordinates'])
    print(record)
    

KeyboardInterrupt: 

In [2]:
import geopandas as gpd
import json
from shapely.geometry import Point

# 1. 加载SA2边界数据
fp = "D:/COMP90024_2024_ASMT2_Group12/data/SA2-Map/SA2_2021_AUST_GDA2020.shp"
sa2_gdf = gpd.read_file(fp)

# 投影到一个合适的CRS (如EPSG:3857)
sa2_gdf = sa2_gdf.to_crs(epsg=3857)

# 2. 加载和解析JSON数据
file_path = 'asthma_with_coordinates_filtered.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# 3. 提取坐标并转换为GeoDataFrame
records = []
for feature in data['features']:
    properties = feature['properties']
    coordinates = properties['coordinates']
    lon = coordinates['longitude']
    lat = coordinates['latitude']
    point = Point(lon, lat)
    record = {
        'id': feature['id'],
        'area_code': properties['area_code'],
        'area_name': properties['area_name'],
        'asthma_me_2_rate_3_11_7_13': properties['asthma_me_2_rate_3_11_7_13'],
        'respirtry_me_2_rate_3_11_7_13': properties['respirtry_me_2_rate_3_11_7_13'],
        'coordinates': coordinates,
        'geometry': point
    }
    records.append(record)

# 转换为GeoDataFrame，投影到同一个CRS (EPSG:3857)
gdf = gpd.GeoDataFrame(records, geometry='geometry', crs="EPSG:4326").to_crs(epsg=3857)

# 4. 进行空间连接
gdf_with_sa2 = gpd.sjoin_nearest(gdf, sa2_gdf, how='left', distance_col='distance')

# 5. 添加SA2区域名称并保留坐标
gdf_with_sa2 = gdf_with_sa2[['id', 'area_code', 'area_name', 'asthma_me_2_rate_3_11_7_13', 'respirtry_me_2_rate_3_11_7_13', 'coordinates', 'SA2_NAME21']]

# 6. 转换为JSON格式并保存
output_data = gdf_with_sa2.to_dict(orient='records')

output_file_path = 'asthma_with_sa2.json'
with open(output_file_path, 'w') as json_output_file:
    json.dump(output_data, json_output_file, indent=2)

print(f'Simplified data with SA2 saved to: {output_file_path}')







Simplified data with SA2 saved to: asthma_with_sa2.json
